# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
print(toxic_comments.shape)
toxic_comments.head(10)

(159571, 2)


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Посмотрим, в каком соотношений присутсвуют значения в целевом признаке toxic:

In [4]:
toxic_comments['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [5]:
class_ratio = toxic_comments['toxic'].value_counts()[0] / toxic_comments['toxic'].value_counts()[1]
class_ratio

8.834884437596301

Классы несбалансированы. Отношение ~1:9 

Проведем балансировку классов двумя способами: изменение весов в модели обучения и ресемплирование с уменьшением класса 0, после чего сравним качество

Подготовим признаки и целевой признак перед обучением:

In [6]:
%%time

m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

toxic_comments['lemm_text'] = toxic_comments['text'].apply(lemmatize_text)

toxic_comments = toxic_comments.drop(['text'], axis=1)
del m

CPU times: user 43.8 s, sys: 9.03 s, total: 52.8 s
Wall time: 1min 52s


Разобьем выборку по отношению 60/20/20. Уменьшим количество кроссвалидаций до 3 из-за размера выборки

In [7]:
target = toxic_comments['toxic']
features = toxic_comments.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12082020)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12082020)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values.astype('U'))
features_valid = count_tf_idf.transform(features_valid['lemm_text'].values.astype('U'))
features_test = count_tf_idf.transform(features_test['lemm_text'].values.astype('U'))
print('features_train:', features_train.shape)
print('features_valid:', features_valid.shape)
print('features_test:', features_test.shape)
cv_counts = 3

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


features_train: (95742, 126292)
features_valid: (31914, 126292)
features_test: (31915, 126292)


In [8]:
%%time
classificator = LogisticRegression()
train_f1 = cross_val_score(classificator,
                      features_train,
                      target_train,
                      cv=cv_counts,
                      scoring='f1').mean()
classificator.fit(features_train, target_train)
valid_f1 = f1_score(target_valid, classificator.predict(features_valid))

print('F1 на CV', train_f1)
print('F1 на валидации', valid_f1)

F1 на CV 0.6732278141449742
F1 на валидации 0.7268891446106636
CPU times: user 12.5 s, sys: 12.6 s, total: 25.1 s
Wall time: 25.1 s


#### Изменение весов в модели обучения:

In [15]:
%%time
dict_classes={0:1, 1:class_ratio}
classificator = LogisticRegression(class_weight=dict_classes)
train_f1_ballanced = cross_val_score(classificator,
                                    features_train,
                                    target_train,
                                    cv=cv_counts,
                                    scoring='f1').mean()
classificator.fit(features_train, target_train)
valid_f1_balanced = f1_score(target_valid, classificator.predict(features_valid))

print('F1 на CV с балансированными классами', train_f1_ballanced)
print('F1 на валидации с балансированными классами', valid_f1_balanced)

F1 на CV с балансированными классами 0.7553768002693207
F1 на валидации с балансированными классами 0.7586593745617726
CPU times: user 30.3 s, sys: 26.1 s, total: 56.5 s
Wall time: 56.5 s


In [16]:
%%time

classificator = LogisticRegression(class_weight='balanced')
train_f1_balanced = cross_val_score(classificator, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 на CV с балансированными классами', train_f1_ballanced)

F1 на CV с балансированными классами 0.7553768002693207
CPU times: user 17.5 s, sys: 15.7 s, total: 33.1 s
Wall time: 33.2 s


Как видно на обучающей выборке F1-мера увеличилась. Встроенный метод повторяет значение F1.

#### Ресемплирование с уменьшением класса 0

Сделаем количество записей с классом 0 таким же, как и количество записей с классом 1

In [11]:
toxic_comments_train = toxic_comments.iloc[target_train.index]

target_train_class_zero = toxic_comments_train[toxic_comments_train['toxic'] == 0]['toxic']
target_train_class_one = toxic_comments_train[toxic_comments_train['toxic'] == 1]['toxic']

In [12]:
target_train_class_zero_downsample = target_train_class_zero.sample(target_train_class_one.shape[0],
                                                                    random_state=12345)
target_train_downsample = pd.concat([target_train_class_zero_downsample, target_train_class_one])

features_train_downsample = toxic_comments.iloc[target_train_downsample.index]
features_train_downsample, target_train_downsample = shuffle(features_train_downsample,
                                                             target_train_downsample,
                                                             random_state=12345)
features_train_downsample = count_tf_idf.transform(features_train_downsample['lemm_text']
                                                   .values.astype('U'))
del count_tf_idf
del stopwords

In [17]:
classificator = LogisticRegression()
train_f1_downsampled = cross_val_score(classificator,
                      features_train_downsample,
                      target_train_downsample,
                      cv=cv_counts,
                      scoring='f1').mean()
classificator.fit(features_train_downsample,target_train_downsample)
valid_f1_downsampled = f1_score(target_valid, classificator.predict(features_valid))

print('F1 на CV с уменьшением классов', train_f1_downsampled)
print('F1 на валидации с уменьшением классов', valid_f1_downsampled)

F1 на CV с уменьшением классов 0.8821013380800937
F1 на валидации с уменьшением классов 0.6953144575294412


Значение F1-меры существенно увеличилось

In [18]:
index = ['LogisticRegression',
         'LR balansed classes',
         'LR downsampled classes']
data = {'F1 на CV':[train_f1,
                    train_f1_balanced,
                    train_f1_downsampled],
        'F1 на валидации':[valid_f1,
                           valid_f1_balanced,
                           valid_f1_downsampled]}

scores_data = pd.DataFrame(data=data, index=index)
scores_data

,F1 на CV,F1 на валидации
LogisticRegression,0.673228,0.726889
LR balansed classes,0.746746,0.758659
LR downsampled classes,0.882101,0.695314


По полученым результатом оптимальным показателем для F1 обладает классификатор, где учтен вес классов. Удалим ненужные переменные.

In [19]:
del toxic_comments_train
del target_train_class_zero
del target_train_class_one
del target_train_class_zero_downsample
del target_train_downsample
del features_train_downsample

На тестировании оптимальным показателем для F1 обладает классификатор, где учтен вес классов. В обучении мы будем использовать именно этот метод балансирования.

## Обучение

Исследуем следующие модели для обучения:

- LogisticRegression
- DecisionTreeClassifier
- CatBoostClassifier

#### LogisticRegression

In [16]:
%%time

classificator = LogisticRegression()
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':[dict_classes]}]


clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)
print("Best parameters for LogisticRegression:")
print()
LR_best_params = clf.best_params_
print(LR_best_params)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f  for %r" % (mean, params))
print()

cv_f1_LR = max(means)

Best parameters for LogisticRegression:

{'C': 10, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'lbfgs'}

Grid scores on development set:

0.713  for {'C': 0.1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'newton-cg'}
0.713  for {'C': 0.1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'lbfgs'}
0.713  for {'C': 0.1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'liblinear'}
0.755  for {'C': 1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'newton-cg'}
0.755  for {'C': 1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'lbfgs'}
0.755  for {'C': 1, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'liblinear'}
0.763  for {'C': 10, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'newton-cg'}
0.763  for {'C': 10, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'lbfgs'}
0.763  for {'C': 10, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'liblinear'}

CPU times: user 5min 43s, sys: 5min 37s, total: 11m

In [17]:
%%time

classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
valid_f1_LR = f1_score(target_valid, target_predict)

print('F1 на cv для LogisticRegression', cv_f1_LR)
print('F1 на валидации для LogisticRegression', valid_f1_LR)

F1 на cv для LogisticRegression 0.7631191168916892
F1 на валидации для LogisticRegression 0.7647146475935032
CPU times: user 25 s, sys: 25.5 s, total: 50.5 s
Wall time: 50.6 s


#### DecisionTreeClassifier

In [23]:
%%time

classificator = DecisionTreeClassifier()
hyperparams = [{'max_depth':[x for x in range(30,51,2)],
                'random_state':[12345],
                'class_weight':[dict_classes]}]


clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=cv_counts)
clf.fit(features_train, target_train)
print("Best parametrs for DecisionTreeClassifier:")
print()
DTC_best_params = clf.best_params_
print(DTC_best_params)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f for %r"% (mean, params))
print()

cv_f1_DTC = max(means)

Best parametrs for DecisionTreeClassifier:

{'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 40, 'random_state': 12345}

Grid scores on development set:

0.605 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 30, 'random_state': 12345}
0.609 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 32, 'random_state': 12345}
0.615 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 34, 'random_state': 12345}
0.615 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 36, 'random_state': 12345}
0.620 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 38, 'random_state': 12345}
0.627 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 40, 'random_state': 12345}
0.621 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 42, 'random_state': 12345}
0.606 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 44, 'random_state': 12345}
0.605 for {'class_weight': {0: 1, 1: 8.834884437596301}, 'm

In [24]:
%%time

classificator = DecisionTreeClassifier()
classificator.set_params(**DTC_best_params)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
valid_f1_DTC = f1_score(target_valid, target_predict)
print('F1 на cv для DecisionTreeClassifier', cv_f1_DTC)
print('F1 на валидации для DecisionTreeClassifier', valid_f1_DTC)

F1 на cv для DecisionTreeClassifier 0.6266521659986855
F1 на валидации для DecisionTreeClassifier 0.604064678831034
CPU times: user 35.3 s, sys: 0 ns, total: 35.3 s
Wall time: 35.4 s


#### CatBoostClassifier

In [20]:
%%time

classificator = CatBoostClassifier(verbose=False, iterations=200)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
cv_f1_CBC = cross_val_score(classificator,
                                         features_train, 
                                         target_train, 
                                         cv=cv_counts, 
                                         scoring='f1').mean()
valid_f1_CBC = f1_score(target_valid, target_predict)
print('F1 на cv для CatBoostClassifier', cv_f1_CBC)
print('F1 на валидации для CatBoostClassifier', valid_f1_CBC) 

F1 на cv для CatBoostClassifier 0.7186686644372537
F1 на валидации для CatBoostClassifier 0.7413539367181752
CPU times: user 49min 5s, sys: 8min 8s, total: 57min 14s
Wall time: 57min 23s


Выведем таблицу валидации исследованных моделей:

In [21]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier']
data = {'F1 на CV':[cv_f1_LR,
                    cv_f1_DTC,
                    cv_f1_CBC],
        'F1 на валидации':[valid_f1_LR,
                           valid_f1_DTC,
                           valid_f1_CBC]}

scores_data = pd.DataFrame(data=data, index=index)
scores_data

,F1 на CV,F1 на валидации
LogisticRegression,0.763119,0.764715
DecisionTreeClassifier,0.625155,0.605458
CatBoostClassifier,0.718669,0.741354


Наивысшим показателем метрики F1 обладает модель LogisticRegression. Проведем тестирование данной модели: 

In [22]:
classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
predict_test = classificator.predict(features_test)
print('Метрики LogisticRegression:')
print('F1:', f1_score(target_test, predict_test))
print('Accuracy:', accuracy_score(target_test, predict_test))
print()

Метрики LogisticRegression:
F1: 0.7650257163850109
Accuracy: 0.94989816700611



## Выводы

В ходе работы над проектом было сделано:

- В ходе преподготовки были получены признаки для обучения и разделена выборка на обучающую, валидационную и тестовую.
- На тестировании оптимальным показателем для F1 обладает классификатор, где учтен вес классов. Для обучения использовали именно этот метод балансирования.
- Обучили модели:
    - LogisticRegression
    - DecisionTreeClassifier
    - CatBoostClassifier 
и  для кажой определили лучшие параметры качества.

Исходные данные обладают большим количеством признаков. Созданных столбцов больше, чем записей данных. Так как TF-IDF превращают текст в численные значения, лучшей моделью стала LogisticRegression.

На тестовой выбоке метрика F1 у LogisticRegression получилась равной 0.765. Также у данной модели хорошее значение метрики Accuracy - 0.95, это говорит нам, что токсичные комментарии находятся лучше.